In [ ]:
import requests
import os
from dotenv import load_dotenv

load_dotenv()
# Retrieve API key and endpoints from environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AI_SEARCH_ENDPOINT = os.getenv("AI_SEARCH_ENDPOINT")
AI_SEARCH_SEMANTIC = os.getenv("AI_SEARCH_SEMANTIC")
AI_SEARCH_KEY = os.getenv("AI_SEARCH_KEY")
AI_SEARCH_INDEX = os.getenv("AI_SEARCH_INDEX")

def request_gpt(prompt):
    
    endpoint = AZURE_OPENAI_ENDPOINT
    headers = {
        "Content-Type": "application/json",
        "api-key": AZURE_OPENAI_API_KEY
    }
    payload = {
        "messages": [
            {
                "role": "system",
                "content": f"""
                    You are an assistant for medical professionals. 
                    When asked about a disease, search the "disease" section of the provided data. If found, answer based on the document. 
                    If not, inform the user that no document is available for that disease.
                """
            },
            {
                "role": "user",
                "content": prompt
            }
        ],
        "temperature": 0.1,
        "top_p": 0.6,
        "max_tokens": 800,
        "data_sources": [
        {
            "type": "azure_search",
            "parameters": {
                "endpoint": AI_SEARCH_ENDPOINT,
                "semantic_configuration": AI_SEARCH_SEMANTIC,
                "fields_mapping": {
                    "content_fields_separator": "\n",
                    "content_fields": None,
                    "filepath_field": None,
                    "title_field": "title",
                    "url_field": None,
                    "vector_fields": [
                    "text_vector"
                    ]
                },
                "in_scope": True,
                "query_type": "semantic",
                "strictness": 4,
                "top_n_documents": 5,
                "authentication": {
                    "type": "api_key",
                    "key": AI_SEARCH_KEY,
                },                
                "index_name": AI_SEARCH_INDEX
            }
        }
        ]
    }
    
    response = requests.post(endpoint, headers=headers, json=payload)
        
    if response.status_code == 200:
        response_json = response.json()
        content = response_json["choices"][0]["message"]["content"]
        if content == "The requested information is not available in the retrieved data. Please try another query or topic.":
            return content, None    
    
        if response_json["choices"][0]["message"]["context"]:            
            citations = response_json["choices"][0]["message"]["context"]["citations"]
            formatted_citation_list = list()
            i = 0
            for c in citations:
                i += 1
                temp = f"<details><summary>Citation{i}</summary><ul>{c['content']}</ul></details>"
                formatted_citation_list.append(temp)
        return content, "".join(formatted_citation_list)
    else:
        return f"{response.status_code}, {response.text}"

prompt = "의 치료 가이드라인을 알려줘"
request_gpt(prompt)
